In [1]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 24.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_6

In [2]:
# resolving "No inf checks were recorded for this optimizer." issue
!pip uninstall torch -y
!pip install torch==2.1

Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Successfully uninstalled torch-2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.0
    Uninstalling triton-2.3.0:
      Successfully uninstalled triton-2.3.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully uninstalled nvidia-nccl-cu12-2.20.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.1.0 which is incompatible.
torchtext 0.18.0

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
from sklearn.model_selection import train_test_split

In [4]:
# Load model

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4481: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
# Using Base Model

model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
question = "What is MDR-GNB?"
prompt=f'''[INST] {question} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] What is MDR-GNB? [/INST] MDR-GNB stands for "Medical Device Regulation - General Notified Body." A Notified Body (NB) is an independent third-party organization that is designated by a European Union (EU) Member State to carry out conformity assessment activities under EU legislation. In the context of medical devices, an NB is responsible for assessing the conformity of certain medical devices with the essential requirements of the Medical Devices Regulation (MDR) and issuing a certificate of conformity. The MDR-GNB specifically refers to a Notified Body that operates under the MDR. The MDR is a regulation that sets out the requirements for placing medical devices on


In [7]:
# Prompt Engineering

intstructions_string = f"""MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to questions aptly and ends responses with its signature '–MedGPT'. \
MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following question.
"""
prompt_template = lambda question: f'''[INST] {intstructions_string} \n{question} \n[/INST]'''

prompt = prompt_template(question)
print(prompt)

[INST] MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to questions aptly and ends responses with its signature '–MedGPT'. MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following question.
 
What is MDR-GNB? 
[/INST]


In [8]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] MedGPT, functioning as a virtual data science consultant on clinical questions, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to questions aptly and ends responses with its signature '–MedGPT'. MedGPT will tailor the length of its responses to match the client's question, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following question.
 
What is MDR-GNB? 
[/INST] MDR-GNB stands for Multidrug-Resistant Gram-Negative bacteria. These are types of bacteria that have developed resistance to multiple antibiotics, making infections caused by them harder to treat. Gram-negative bacteria have a unique cell wall structure that makes it more difficult for antibiotics to penetrate and kill the bacteria compared to Gram-positive bacteria. The multidrug resistance in MDR-GNB makes these infections a significant public health conc

In [9]:
# Prepare Model for Training

model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [10]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [11]:
# Preparing Training Dataset

import os
import xml.etree.ElementTree as ET

class XMLParentMap:
    def __init__(self, tree):
        self.tree = tree
        self.parent_map = self.build_parent_map()

    def build_parent_map(self):
        parent_map = {}
        for p in self.tree.iter():
            for c in p:
                if c in parent_map:
                    parent_map[c].append(p)
                else:
                    parent_map[c] = [p]
        return parent_map

    def get_all_parents(self, element):
        all_parents = []
        self._collect_all_parents(element, all_parents)
        return all_parents

    def _collect_all_parents(self, element, all_parents):
        if element in self.parent_map:
            parents = self.parent_map[element]
            for parent in parents:
                all_parents.append(parent)
                self._collect_all_parents(parent, all_parents)

In [12]:
class XMLDirectoryReader:
    def __init__(self, directory):
        self.directory = directory

    def load_data(self):
        documents = []
        for filename in os.listdir(self.directory):
            if filename.endswith(".xml"):
                file_path = os.path.join(self.directory, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    tree = ET.parse(file)
                    root = tree.getroot()
                    parent_map_instance = XMLParentMap(tree)
                    text_content = self.extract_text_from_xml(root, parent_map_instance)
                    if text_content:  # Vérifie que le texte n'est pas vide
                        documents.append(text_content)
        return documents

    def extract_text_from_xml(self, root, parent_map):
        namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
        texts = []
        for elem in root.iter():
            all_parents = parent_map.get_all_parents(elem)

            # Vérifier si l'élément ou un de ses parents est dans <biblStruct>
            if any(p.tag in ['{http://www.tei-c.org/ns/1.0}biblStruct', '{http://www.tei-c.org/ns/1.0}figure'] for p in all_parents):
                continue  # Passer à l'élément suivant si dans <biblStruct>


            # Exclure certains éléments spécifiques (comme les métadonnées ou les informations de publication)
            if elem.tag in ['{http://www.tei-c.org/ns/1.0}title', '{http://www.tei-c.org/ns/1.0}head', '{http://www.tei-c.org/ns/1.0}p']:
                if elem.text and elem.text.strip():  # Exclure les éléments avec du texte vide
                    text = elem.text.strip()
                    if elem.tag == '{http://www.tei-c.org/ns/1.0}title':
                        text = f"\n\ntitle : {text}"  # Ajouter le préfixe "title :"
                    elif elem.tag == '{http://www.tei-c.org/ns/1.0}head':
                        text = f"\nhead : {text}"  # Ajouter le préfixe "head :"
                    texts.append(text)
        return "\n".join(texts)

In [44]:
data = XMLDirectoryReader("data/Medical_Guidelines/XML").load_data()

In [45]:
print(len(data))

2


In [63]:
# Fonction pour segmenter le texte en utilisant les titres et les sous-titres
def segment_text(data):
    segments = []
    current_segment = []

    for text in data:
        lines = text.split("\n")
        for line in lines:
            if line.startswith("title :") or line.startswith("head :"):
                if current_segment:
                    segments.append("\n".join(current_segment).strip())
                    current_segment = []
            current_segment.append(line)

        if current_segment:
            segments.append("\n".join(current_segment).strip())

    return segments

# Segmentation du document
segments = segment_text(data)

# Diviser les segments en ensembles d'entraînement et de validation
train_segments, val_segments = train_test_split(segments, test_size=0.2, random_state=42)

# Affichage des segments pour vérification
#for i, segment in enumerate(segments):
#    print(f"Segment {i+1}:\n{segment}\n{'-'*40}\n")

print(len(train_segments))
print(len(val_segments))

72
19


In [72]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples

    # tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    # Convertir les array en listes
    tokenized_inputs = {key: value[0].tolist() for key, value in tokenized_inputs.items()}

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_train_data = [tokenize_function(segment) for segment in train_segments]
tokenized_test_data = [tokenize_function(segment) for segment in val_segments]

In [73]:
print(len(tokenized_train_data))
print(tokenized_train_data[1])

72
{'input_ids': [1, 1335, 714, 2364, 28726, 377, 269, 366, 28733, 411, 11143, 9810, 598, 2441, 2230, 13, 24994, 28705, 28750, 28723, 28740, 28747, 10934, 6883, 347, 4921, 286, 354, 24167, 377, 269, 366, 28733, 411, 11143, 9810, 598, 2441, 2230, 325, 9014, 28731, 1159, 13301, 28804], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [74]:
from datasets import Dataset

train_dataset = Dataset.from_dict(tokenized_train_data)
test_dataset = Dataset.from_dict(tokenized_test_data)

AttributeError: 'list' object has no attribute 'items'

In [75]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [76]:
# Fine-tuning Model

# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "Finetuned_Med_GPT",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [77]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    args=training_args,
    data_collator=data_collator
)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,2.706500,2.639608
2,2.356600,2.409013
4,2.343600,2.317449
6,2.180800,2.267511
8,1.863000,2.250223


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=T